# .csv to PSQL 
- connect to the postgreSQL with a .gitignore file to hide your login credentials

1) create the initialization file:

use your terminal or notepad, unless your favorite editor directly converts to this format .ini file.
on Mac
touch database_file_init.ini
vi database_file.ini 
[postgresql]
host=localhost
database=what_databse_you_want_to_access
user= some_user_you_created_for_this_user_in_psql
password= some_password_you_have_to_this_db
------------------------------------------------

2) config.py file this will be used to take data from init file and outputs a dictionary.

from configparser import ConfigParser

def config(filename='database.ini', section='postgresql'):

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}

# Checks to see if section (postgresql) parser exists
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]

# Returns an error if a parameter is called that is not listed in the initialization file
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

return db


In [1]:
import psycopg2             # python->psql connection
import psycopg2.extras
import pandas as pd         # create dataframes 
import os                   # fetch files
from config import config

# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

In [3]:
def create_staging_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS staging_fake_ppl;
        CREATE UNLOGGED TABLE staging_fake_ppl (
            credit_card         TEXT,
            email               TEXT,
            first_name          TEXT,
            last_name           TEXT,
            primary_phone       TEXT
        );""")
with conn.cursor() as cursor:
    create_staging_table(cursor)

DependentObjectsStillExist: cannot drop table staging_fake_ppl because other objects depend on it
DETAIL:  view customer_cpu_sales depends on table staging_fake_ppl
HINT:  Use DROP ... CASCADE to drop the dependent objects too.


In [3]:
addr_df = pd.read_csv('data/user.csv')

addr_=addr_df.iloc[:,1:]
addr_.head()

,credit_card,email,first_name,last_name,primary_phone_number
0,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
1,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
2,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
3,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
4,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348


In [18]:
# #Import csv
# def os_dir_search(file):
#     u=[]
#     for p,n,f in os.walk(os.getcwd()):
        
#         for a in f:
#             a = str(a)
#             if a.endswith(file): # can be (.csv) or a file like I did and search 
#                 print(a)
#                 print(p)
#                 t=pd.read_csv(p+'/'+file,names=['row_id','credit_card',
#                                                 'email','first_name','last_name','primary_phone'],header=0)
            
#     return t

# # need to use (.csv,.png, etc) because it is looking by file type ending


# addr_=os_dir_search('user.csv')
# addr_.head()

user.csv
/Users/daheekim/Desktop/VisualStudio/psycopg2/data


,row_id,credit_card,email,first_name,last_name,primary_phone
0,1,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
1,2,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
2,3,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
3,4,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
4,5,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348


In [19]:
# csv file to dictionary
s_=addr_.to_dict('records')
s_[:2]

[{'row_id': 1,
  'credit_card': '5399-3484-4724-7187',
  'email': 'gso@qiegan.sqe',
  'first_name': 'Donyell Ann',
  'last_name': 'Ospina',
  'primary_phone': 5219459148},
 {'row_id': 2,
  'credit_card': '1630-5261-6108-7631',
  'email': 'xnji@gfruaxqnvm.fha',
  'first_name': 'Bishop',
  'last_name': 'Siyed',
  'primary_phone': 4164254716}]

In [21]:
# 
addr_.to_csv('data/address_Python_convertR.csv',index=False)

In [10]:
# Another way of reading table
import pandas.io.sql as sqlio
sql = "select * from staging_fake_ppl limit 4;"
dat = sqlio.read_sql_query(sql, conn)
conn = None
dat

,credit_card,email,first_name,last_name,primary_phone
0,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
1,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
2,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
3,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561


In [4]:
def create_staging_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS staging_fake_ppl02;
        CREATE UNLOGGED TABLE staging_fake_ppl02 (
            credit_card         TEXT,
            email               TEXT,
            first_name          TEXT,
            last_name           TEXT,
            primary_phone       TEXT
        );""")

conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)

with conn.cursor() as cursor:
    create_staging_table_(cursor)

In [5]:

def fcn(df,table,cur):
#     df=addr_df_
#     table='staging_fake_ppl02'
    # df is the dataframe
    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

In [6]:
fcn(addr_,'staging_fake_ppl02',cur)
cur.close()
conn.close()

UndefinedColumn: column "primary_phone_number" of relation "staging_fake_ppl02" does not exist
LINE 1: ...ake_ppl02 (credit_card,email,first_name,last_name,primary_ph...
                                                             ^
